In [ ]:
import sys
import os
print(sys.executable)


c:\Users\Dev\Desktop\OpenClassrooms\Project11\RAG\Events-with-RAG\.venv\Scripts\python.exe


In [148]:
%pip list


Package                   Version
------------------------- -----------
aiohappyeyeballs          2.6.1
aiohttp                   3.12.11
aiosignal                 1.3.2
alembic                   1.16.1
amqp                      5.3.1
annotated-types           0.7.0
anyio                     4.9.0
asttokens                 3.0.0
async-timeout             4.0.3
attrs                     25.3.0
babel                     2.17.0
bcrypt                    4.3.0
beautifulsoup4            4.13.4
bs4                       0.0.2
cachetools                6.0.0
certifi                   2025.4.26
cffi                      1.17.1
charset-normalizer        3.4.2
colorama                  0.4.6
comm                      0.2.2
croniter                  6.0.0
cryptography              45.0.3
dataclasses-json          0.6.7
debtcollector             3.0.0
debugpy                   1.8.14
decorator                 5.2.1
dnspython                 2.7.0
dogpile.cache             1.4.0
eval_type_backport 

You should consider upgrading via the 'c:\Users\Dev\Desktop\OpenClassrooms\Project11\RAG\Events-with-RAG\.venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [149]:
import langchain
import faiss
import mistral  # ou mistralai si besoin
from datetime import datetime

print("Toutes les bibliothèques sont bien importées ✅")


Toutes les bibliothèques sont bien importées ✅


In [150]:
cutoffDate = datetime.now().strftime("%Y-%m-%dT00:00:00")
cutoffDate = cutoffDate[0:3] + str(int(cutoffDate[3])-1) + cutoffDate[4:]
cutoffDate

'2024-06-20T00:00:00'

In [179]:
import requests
import pandas as pd

base_url = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/evenements-publics-openagenda/records"

all_records = []

# 100 pages of 100 = 10000 results
for offset in range(0, 10000, 100):
    params = {
        "limit": 100,
        "where": "location_region = 'Nouvelle-Aquitaine' AND lastdate_end >= '" + cutoffDate + "'",
        "offset": offset,
    }

    response = requests.get(base_url, params=params)
    response.raise_for_status()
    data = response.json()
    records = data.get("results", [])

    if not records:
        break  # Stop if no more records returned

    all_records.extend(records)



In [183]:
data

{'total_count': 6918, 'results': []}

In [152]:
# Convert to DataFrame
df = pd.DataFrame.from_records(all_records)

len(df)

6918

In [153]:
cols_to_convert = [
    "updatedat", 
    "firstdate_begin", 
    "firstdate_end",
    "lastdate_begin",
    "lastdate_end",
    ] 

for col in cols_to_convert:
    df[col] = pd.to_datetime(df[col], format='ISO8601')


In [154]:
df.columns

Index(['uid', 'slug', 'canonicalurl', 'title_fr', 'description_fr',
       'longdescription_fr', 'conditions_fr', 'keywords_fr', 'image',
       'imagecredits', 'thumbnail', 'originalimage', 'updatedat',
       'daterange_fr', 'firstdate_begin', 'firstdate_end', 'lastdate_begin',
       'lastdate_end', 'timings', 'accessibility', 'accessibility_label_fr',
       'location_uid', 'location_coordinates', 'location_name',
       'location_address', 'location_district', 'location_insee',
       'location_postalcode', 'location_city', 'location_department',
       'location_region', 'location_countrycode', 'location_image',
       'location_imagecredits', 'location_phone', 'location_website',
       'location_links', 'location_tags', 'location_description_fr',
       'location_access_fr', 'attendancemode', 'onlineaccesslink', 'status',
       'age_min', 'age_max', 'originagenda_title', 'originagenda_uid',
       'contributor_email', 'contributor_contactnumber',
       'contributor_contactnam

In [155]:
# Preview
print(f"Fetched {len(df)} records.")
df[["uid", "slug", "description_fr", "location_region", "lastdate_end"]].head()


Fetched 6918 records.


,uid,slug,description_fr,location_region,lastdate_end
0,81494288,visite-guidee-de-collonges-la-rouge,Laissez-vous conter l’histoire et les légendes...,Nouvelle-Aquitaine,2024-09-22 14:30:00+00:00
1,38171454,visita-guiada-sobre-el-traje-a-traves-de-la-hi...,Le musée retrace l'évolution du costume à trav...,Nouvelle-Aquitaine,2024-09-21 16:00:00+00:00
2,98956271,ateliers-dart-floral-5792915,Apprendre à réaliser des compositions florales...,Nouvelle-Aquitaine,2024-06-27 16:00:00+00:00
3,69227970,domaine-doronozia,RH,Nouvelle-Aquitaine,2032-01-01 03:00:00+00:00
4,61334124,decouverte-de-la-salamandre,Dans le cadre des RV durables,Nouvelle-Aquitaine,2024-10-25 19:00:00+00:00


In [156]:
from bs4 import BeautifulSoup
from bs4 import MarkupResemblesLocatorWarning
import warnings
warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

def clean_html(text):
    if isinstance(text, str):
        return BeautifulSoup(text, "html.parser").get_text(separator="")
    return ""

df["longdescription_fr"] = df["longdescription_fr"].apply(clean_html)

In [157]:
df.to_csv("output/clean_evenements_publics_openagenda.csv", index=False)

In [176]:
import os
import json

metrics = {
    "cutoffDate": cutoffDate,
    "required_columns": ["title_fr", "description_fr", "timings"],
    "row_count": len(df),
}

file_path = os.path.join("tests/", f"expected_metrics.json")



with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(metrics, f, indent=4, ensure_ascii=False)

In [184]:
import json
from datetime import datetime
from dateutil import parser  # Robust parsing for ISO 8601
import locale
from datetime import datetime
from babel.dates import format_datetime

# Optionnel : utile si on ne veut pas dépendre de la configuration système
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')  # Peut ne pas fonctionner sur Windows sans les locales installées



def parse_dates(dates_str):
    try:
        parsed = json.loads(dates_str)
        return [
            {
                "begin": parser.parse(item["begin"]),
                "end": parser.parse(item["end"])
            }
            for item in parsed
        ]
    except (json.JSONDecodeError, KeyError, TypeError):
        return []

def format_dates_fr(dates):
    return [
        f"{format_datetime(d['begin'], 'EEEE d MMMM yyyy, HH:mm', locale='fr_FR')} – {format_datetime(d['end'], 'EEEE d MMMM yyyy, HH:mm', locale='fr_FR')}"
        for d in dates
    ]


In [195]:
def build_documents_from_df(df):
    documents = []
    for _, row in df.iterrows():
        raw_dates = row.get("timings", "")
        parsed_dates = parse_dates(raw_dates)
        display_dates = format_dates_fr(parsed_dates)
        doc = {
            "titre": row.get("title_fr", "") or "",
            "description": row.get("description_fr", "") or "",
            "longue description": row.get("longdescription_fr", "") or "",
            "adresse": ", ".join(filter(None, [
                # row.get("location_name", ""),
                row.get("location_address", "")
            ])),
            "telephone": row.get("location_phone", "") or "",
            "site Web": row.get("location_website", "") or "",
            "dates": parsed_dates,               # structuree, plus facile à manipuler
            "dates affichage": display_dates,       # claires, pour les réponses
            "prix": row.get("conditions_fr", "") or "",
        }
        documents.append(doc)
    return documents


In [196]:
docs = build_documents_from_df(df)

In [194]:
docs[10]

{'titre': 'Festival Musique et Patrimoine en Vienne et Gartempe',
 'description': 'Orchestre ANIMA - Vivaldi 4 Saisons',
 'longue description': 'Anima est un orchestre symphonique fondé en 2022 à l’initiative de deux jeunes musiciens.Au programme, ce chef-d’œuvre de son traité musical « La confrontation entre l’harmonie et l’invention » dédié aux quatre saisons (printemps, été, automne et hiver), le célèbre prêtre-maestro italien Antonio Vivaldi (1678-1741), alors âgé de 47 ans,a composé cet hymne universel grandiose à la nature avec tout le génie de son langage musical virtuose expressif.',
 'adresse': 'Eglise Saint-Martin à Saint-Martin-la-Rivière, 86300 Valdivienne',
 'telephone': '',
 'site Web': '',
 'dates': [{'begin': datetime.datetime(2024, 7, 22, 20, 30, tzinfo=tzoffset(None, 7200)),
   'end': datetime.datetime(2024, 7, 22, 22, 0, tzinfo=tzoffset(None, 7200))}],
 'dates affichage': ['lundi 22 juillet 2024, 20:30 – lundi 22 juillet 2024, 22:00'],
 'prix': "TARIF PLEIN : 17€\nTA

In [162]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("dangvantuan/sentence-camembert-base")

In [197]:
def get_embedding_text(doc):
    return "\n".join([
        doc.get("titre", ""),
        doc.get("description", ""),
        # doc.get("longue description", ""),
        "Adresse : " + doc.get("adresse", ""),
        "Dates : " + ", ".join(doc.get("dates affichage", [])),
        # "Prix : " + doc.get("prix", ""),
    ])


In [199]:
corpus = [get_embedding_text(doc) for doc in docs]
for i in range(10):
    print(corpus[i])
    print()

Explorez les rues de Collonges-la-Rouge accompagnés d'un guide !
Laissez-vous conter l’histoire et les légendes surprenantes de Collonges-la-Rouge au cours d'une visite guidée gratuite dans les ruelles de la ville.
Adresse : 1274 Route de Brive, 19500 Collonges-la-Rouge
Dates : dimanche 22 septembre 2024, 10:30 – dimanche 22 septembre 2024, 12:00, dimanche 22 septembre 2024, 15:00 – dimanche 22 septembre 2024, 16:30

Visite guidée sur le costume à travers l'histoire de France
Le musée retrace l'évolution du costume à travers l'histoire de France sur treize scènes. La visite commence donc avec la scène de l'Antiquité, l'Empire gallo-romain et se termine par celle des...
Adresse : 20 rue du Palais de Justice, 33340 Lesparre-Médoc
Dates : vendredi 20 septembre 2024, 14:00 – vendredi 20 septembre 2024, 18:00, samedi 21 septembre 2024, 14:00 – samedi 21 septembre 2024, 18:00

Ateliers d'art floral
Apprendre à réaliser des compositions florales modernes et créatives autour d'un thème spécifi

In [ ]:
import faiss
import numpy as np

# Step 1: Create corpus
corpus = [get_embedding_text(doc) for doc in docs]

# Step 2: Encode with normalization (L2 norm = 1)
embeddings = model.encode(corpus, show_progress_bar=True, normalize_embeddings=True)

# Step 3: Use FAISS Index for inner product (cosine similarity when vectors are normalized)
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension) 
index.add(np.array(embeddings))

# Store the original docs for retrieval later
id_to_doc = {i: doc for i, doc in enumerate(docs)}


Batches:  88%|████████▊ | 192/217 [55:36<05:07, 12.29s/it] 

In [165]:
print(corpus[5500])

Découverte du musée des vieux outils de Brie
Visite commentée, métier par métier. Puis libre circulation et réponses aux questions.
Dates : dimanche 22 septembre 2024, 10:00 – 12:00


In [166]:
corpus[0]

"Explorez les rues de Collonges-la-Rouge accompagnés d'un guide !\nLaissez-vous conter l’histoire et les légendes surprenantes de Collonges-la-Rouge au cours d'une visite guidée gratuite dans les ruelles de la ville.\nDates : dimanche 22 septembre 2024, 10:30 – 12:00, dimanche 22 septembre 2024, 15:00 – 16:30"

In [ ]:
def search_documents(query, k=5):
    query_embedding = model.encode([query], normalize_embeddings=True)
    distances, indices = index.search(query_embedding, k)
    results = [id_to_doc[idx] for idx in indices[0]]
    return results


In [ ]:
def format_for_chatbot(doc):
    return (
        f"**{doc['titre']}**\n"
        f"{doc['description']}\n\n"
        # f"{doc['longue description']}\n\n"
        f"📍 Adresse : {doc['adresse']}\n"
        f"📅 Dates : {', '.join(doc['dates affichage'])}\n"
        # f"💰 Prix : {doc['prix']}\n"
        # f"📞 Téléphone : {doc['telephone'] or 'Non fourni'}\n"
        # f"🌐 Site Web : {doc['site Web'] or 'Non fourni'}"
    )


In [169]:
query = "Je veux un événement avec dégustation de vin en septembre"
results = search_documents(query)

for doc in results:
    result = format_for_chatbot(doc)


In [170]:
print(format_for_chatbot(results[0]))

**Dégustation au Bureau d'Information Touristique de Saint-Savin**
Dégustation Surprise

📅 Dates : vendredi 2 août 2024, 10:30 – 12:30



In [171]:
query = "manga"
results = search_documents(query)

for doc in results:
    result = format_for_chatbot(doc)


In [172]:
print(format_for_chatbot(results[0]))

**Performance et lecture**
Expérience sonore, immersive et poétique à travers l’aile Bonheur.

📅 Dates : samedi 21 septembre 2024, 15:30 – 16:00, dimanche 22 septembre 2024, 17:15 – 17:45



In [ ]:
import faiss
import json
import numpy as np
import os

def save_project(index, docs, model_name, path="model3"):
    os.makedirs(path, exist_ok=True)
    
    # Save FAISS index
    faiss.write_index(index, os.path.join(path, "index_faiss.bin"))
    
    # Save documents
    with open(os.path.join(path, "documents.json"), "w", encoding="utf-8") as f:
        json.dump(docs, f, ensure_ascii=False, indent=2, default=str)
    
    # Save metadata (model name)
    with open(os.path.join(path, "meta.json"), "w", encoding="utf-8") as f:
        json.dump({"model": model_name}, f, ensure_ascii=False, indent=2)

    print(f"✅ Project saved to folder: {path}")


In [ ]:
save_project(index, docs, "dangvantuan/sentence-camembert-base", path="model3")

✅ Project saved to folder: model2
